# Initial Data Setup

In [153]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [154]:
# Files to Import
all_nba_file = Path('Resources/End of Season Teams.csv')
all_star_file = Path('Resources/All-Star Selections.csv')
stats_file = Path('Resources/Player Per Game.csv')
adv_stats_file = Path('Resources/Advanced.csv')

# Set up DataFrames
all_nba_df = pd.read_csv(all_nba_file)
all_star_df = pd.read_csv(all_star_file)
stats_df = pd.read_csv(stats_file)
adv_stats_df = pd.read_csv(adv_stats_file)

In [155]:
# Review DataFrames
all_nba_df.head()

,season,lg,type,number_tm,player,position,seas_id,player_id,birth_year,tm,age
0,2022,NBA,All-Defense,1st,Giannis Antetokounmpo,NaN,29918,4164,NaN,MIL,27
1,2022,NBA,All-Defense,1st,Jaren Jackson Jr.,NaN,29991,4632,NaN,MEM,22
2,2022,NBA,All-Defense,1st,Marcus Smart,NaN,30183,4275,NaN,BOS,27
3,2022,NBA,All-Defense,1st,Mikal Bridges,NaN,30205,4658,NaN,PHO,25
4,2022,NBA,All-Defense,1st,Rudy Gobert,NaN,30313,4199,NaN,UTA,29


In [156]:
all_star_df.head()

,player,team,lg,season,replaced
0,Bam Adebayo,East,NBA,2024,False
1,Giannis Antetokounmpo,East,NBA,2024,False
2,Paolo Banchero,East,NBA,2024,False
3,Scottie Barnes,East,NBA,2024,False
4,Jaylen Brown,East,NBA,2024,False


In [157]:
stats_df.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,31136,2024,5025,A.J. Green,NaN,SG,24.0,2,NBA,MIL,...,1.000,0.2,0.9,1.0,0.5,0.1,0.1,0.1,0.9,4.3
1,31137,2024,5026,A.J. Lawson,NaN,SG,23.0,2,NBA,DAL,...,0.632,0.4,0.8,1.2,0.5,0.3,0.1,0.4,0.7,3.8
2,31138,2024,5027,AJ Griffin,NaN,SF,20.0,2,NBA,ATL,...,1.000,0.1,0.7,0.8,0.2,0.1,0.1,0.3,0.3,2.1
3,31139,2024,4219,Aaron Gordon,NaN,PF,28.0,10,NBA,DEN,...,0.652,2.4,4.1,6.5,3.2,0.9,0.7,1.5,1.9,13.9
4,31140,2024,4582,Aaron Holiday,NaN,PG,27.0,6,NBA,HOU,...,0.889,0.3,1.4,1.7,1.8,0.5,0.1,0.8,1.6,7.0


In [158]:
adv_stats_df.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,tov_percent,usg_percent,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp
0,31136,2024,5025,A.J. Green,NaN,SG,24.0,2,NBA,MIL,...,3.7,16.3,0.6,0.2,0.8,0.110,0.9,-2.1,-1.2,0.1
1,31137,2024,5026,A.J. Lawson,NaN,SG,23.0,2,NBA,DAL,...,9.7,19.1,0.1,0.2,0.3,0.065,-1.9,-1.3,-3.3,-0.1
2,31138,2024,5027,AJ Griffin,NaN,SF,20.0,2,NBA,ATL,...,11.6,16.1,-0.3,0.0,-0.3,-0.098,-5.5,-3.3,-8.8,-0.2
3,31139,2024,4219,Aaron Gordon,NaN,PF,28.0,10,NBA,DEN,...,11.4,17.9,3.0,2.0,4.9,0.139,0.9,0.1,1.0,1.3
4,31140,2024,4582,Aaron Holiday,NaN,PG,27.0,6,NBA,HOU,...,11.8,16.4,0.9,1.0,1.9,0.093,-1.7,0.5,-1.2,0.2


# Data Cleaning

In [159]:
# Dropping unneeded columns
all_nba_clean = all_nba_df.drop(columns=['lg','number_tm','position','seas_id','player_id','birth_year','tm','age'])

# Getting dummy columns and combining the players seasons so that their multiple awards are shown
all_nba_pivot = pd.pivot_table(all_nba_clean, index=['season','player'], columns='type', aggfunc=lambda x:'Yes').fillna('No')

# Dropping the old awards for defunct leagues
all_nba_pivot_clean = all_nba_pivot.drop(columns=['All-ABA','All-BAA'])
all_nba_pivot_clean.tail(25)

type                         All-Defense All-NBA All-Rookie
season player                                              
2022   Evan Mobley                    No      No        Yes
       Franz Wagner                   No      No        Yes
       Giannis Antetokounmpo         Yes     Yes         No
       Herbert Jones                  No      No        Yes
       Ja Morant                      No     Yes         No
       Jalen Green                    No      No        Yes
       Jaren Jackson Jr.             Yes      No         No
       Jayson Tatum                   No     Yes         No
       Joel Embiid                    No     Yes         No
       Josh Giddey                    No      No        Yes
       Jrue Holiday                  Yes      No         No
       Karl-Anthony Towns             No     Yes         No
       Kevin Durant                   No     Yes         No
       LeBron James                   No     Yes         No
       Luka Dončić                    No     Yes         No
       Marcus Smart                  Yes      No         No
       Matisse Thybulle              Yes      No         No
       Mikal Bridges                 Yes      No         No
       Nikola Jokić                   No     Yes         No
       Pascal Siakam                  No     Yes         No
       Robert Williams               Yes      No         No
       Rudy Gobert                   Yes      No         No
       Scottie Barnes                 No      No        Yes
       Stephen Curry                  No     Yes         No
       Trae Young                     No     Yes         No

In [160]:
# Dropping unneeded columns
all_star_clean = all_star_df.drop(columns=['team','lg','replaced'])
all_star_clean['All Star'] = 'Yes'
all_star_clean.head()

,player,season,All Star
0,Bam Adebayo,2024,Yes
1,Giannis Antetokounmpo,2024,Yes
2,Paolo Banchero,2024,Yes
3,Scottie Barnes,2024,Yes
4,Jaylen Brown,2024,Yes


In [161]:
# Dropping unneeded columns
adv_clean = adv_stats_df.drop(columns=['seas_id','player_id','birth_year','pos','age','experience','lg','tm','g','mp'])
adv_clean.head()

,season,player,per,ts_percent,x3p_ar,f_tr,orb_percent,drb_percent,trb_percent,ast_percent,...,tov_percent,usg_percent,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp
0,2024,A.J. Green,12.1,0.635,0.867,0.063,1.9,10.1,6.1,7.8,...,3.7,16.3,0.6,0.2,0.8,0.110,0.9,-2.1,-1.2,0.1
1,2024,A.J. Lawson,12.7,0.562,0.471,0.224,5.7,10.0,7.8,7.8,...,9.7,19.1,0.1,0.2,0.3,0.065,-1.9,-1.3,-3.3,-0.1
2,2024,AJ Griffin,1.8,0.403,0.733,0.044,1.6,10.2,5.7,3.8,...,11.6,16.1,-0.3,0.0,-0.3,-0.098,-5.5,-3.3,-8.8,-0.2
3,2024,Aaron Gordon,16.6,0.607,0.186,0.379,8.6,14.3,11.5,13.8,...,11.4,17.9,3.0,2.0,4.9,0.139,0.9,0.1,1.0,1.3
4,2024,Aaron Holiday,11.4,0.591,0.535,0.145,1.8,8.9,5.3,15.2,...,11.8,16.4,0.9,1.0,1.9,0.093,-1.7,0.5,-1.2,0.2


In [162]:
# Merging
first_merged_df = pd.merge(stats_df, all_nba_pivot_clean, on=['season','player'], how='left')
second_merged_df = pd.merge(first_merged_df, all_star_clean, on=['season','player'], how='left')
merged_df = pd.merge(second_merged_df, adv_clean, on=['season','player'], how='left')

merged_df = merged_df[merged_df['lg'] == 'NBA']
merged_df.drop(columns=['player','pos','seas_id','player_id','birth_year','lg','tm','fg_per_game','fga_per_game','x3p_per_game','x2p_per_game',
'x2pa_per_game','x2p_percent','ft_per_game','ws_48'], inplace=True)

merged_df.head()

,season,age,experience,g,gs,mp_per_game,fg_percent,x3pa_per_game,x3p_percent,e_fg_percent,...,blk_percent,tov_percent,usg_percent,ows,dws,ws,obpm,dbpm,bpm,vorp
0,2024,24.0,2,39,0.0,9.2,0.438,2.8,0.423,0.621,...,0.5,3.7,16.3,0.6,0.2,0.8,0.9,-2.1,-1.2,0.1
1,2024,23.0,2,28,0.0,8.3,0.471,1.4,0.325,0.547,...,1.1,9.7,19.1,0.1,0.2,0.3,-1.9,-1.3,-3.3,-0.1
2,2024,20.0,2,18,0.0,7.3,0.289,1.8,0.273,0.389,...,0.7,11.6,16.1,-0.3,0.0,-0.3,-5.5,-3.3,-8.8,-0.2
3,2024,28.0,10,54,54.0,31.5,0.557,1.8,0.293,0.585,...,1.9,11.4,17.9,3.0,2.0,4.9,0.9,0.1,1.0,1.3
4,2024,27.0,6,56,1.0,17.3,0.455,3.0,0.410,0.565,...,0.5,11.8,16.4,0.9,1.0,1.9,-1.7,0.5,-1.2,0.2


# Creating the All Star Prediction Model

In [163]:
# Setting up the data sets for each awards models
all_star_data = merged_df.copy()
all_star_data['All Star'].fillna('No', inplace=True)
y = all_star_data['All Star']
X = all_star_data.drop(columns=['season','All-Defense','All-NBA','All-Rookie','All Star']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [171]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 39
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_98 (Dense)            (None, 8)                 320       
                                                                 
 dense_99 (Dense)            (None, 5)                 45        
                                                                 
 dense_100 (Dense)           (None, 1)                 6         
                                                                 


Total params: 371
Trainable params: 371
Non-trainable params: 0
_________________________________________________________________


In [172]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [173]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1093/1093 [==============================] - 1s 478us/step - loss: 0.2023 - accuracy: 0.9314
Epoch 2/100
1093/1093 [==============================] - 1s 500us/step - loss: 0.0710 - accuracy: 0.9723
Epoch 3/100
1093/1093 [==============================] - 1s 463us/step - loss: 0.0658 - accuracy: 0.9732
Epoch 4/100
1093/1093 [==============================] - 1s 466us/step - loss: 0.0635 - accuracy: 0.9741
Epoch 5/100
1093/1093 [==============================] - 1s 473us/step - loss: 0.0620 - accuracy: 0.9746
Epoch 6/100
1093/1093 [==============================] - 1s 497us/step - loss: 0.0609 - accuracy: 0.9751
Epoch 7/100
1093/1093 [==============================] - 1s 473us/step - loss: 0.0600 - accuracy: 0.9758
Epoch 8/100
1093/1093 [==============================] - 0s 443us/step - loss: 0.0594 - accuracy: 0.9761
Epoch 9/100
1093/1093 [==============================] - 0s 454us/step - loss: 0.0586 - accuracy: 0.9762
Epoch 10/100
1093/1093 [==============================]

In [174]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

365/365 - 0s - loss: 0.0583 - accuracy: 0.9768 - 196ms/epoch - 536us/step
Loss: 0.058259256184101105, Accuracy: 0.9768319725990295


# Setting up All-NBA Prediction Model

In [175]:
# Setting up the data sets for each awards models
all_nba_data = merged_df.copy().drop(columns=['All-Defense','All Star','All-Rookie'])
all_nba_data = all_nba_data[(all_nba_data['season'] != 2023) & (all_nba_data['season'] != 2024)]
all_nba_data['All-NBA'].fillna('No', inplace=True)
y = all_nba_data['All-NBA']
X = all_nba_data.drop(columns=['All-NBA','season']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [180]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 39
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_104 (Dense)           (None, 8)                 320       
                                                                 
 dense_105 (Dense)           (None, 5)                 45        
                                                                 
 dense_106 (Dense)           (None, 1)                 6         
                                                                 
Total params: 371
Trainable params: 371
Non-trainable params: 0
_________________________________________________________________


In [181]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [178]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1042/1042 [==============================] - 1s 459us/step - loss: 0.1249 - accuracy: 0.9660
Epoch 2/100
1042/1042 [==============================] - 0s 462us/step - loss: 0.0392 - accuracy: 0.9853
Epoch 3/100
1042/1042 [==============================] - 0s 459us/step - loss: 0.0343 - accuracy: 0.9870
Epoch 4/100
1042/1042 [==============================] - 0s 461us/step - loss: 0.0326 - accuracy: 0.9875
Epoch 5/100
1042/1042 [==============================] - 0s 457us/step - loss: 0.0312 - accuracy: 0.9877
Epoch 6/100
1042/1042 [==============================] - 0s 458us/step - loss: 0.0301 - accuracy: 0.9882
Epoch 7/100
1042/1042 [==============================] - 1s 491us/step - loss: 0.0297 - accuracy: 0.9883
Epoch 8/100
1042/1042 [==============================] - 0s 460us/step - loss: 0.0292 - accuracy: 0.9888
Epoch 9/100
1042/1042 [==============================] - 0s 463us/step - loss: 0.0284 - accuracy: 0.9885
Epoch 10/100
1042/1042 [==============================]

In [179]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

348/348 - 0s - loss: 0.0351 - accuracy: 0.9878 - 187ms/epoch - 538us/step
Loss: 0.035108231008052826, Accuracy: 0.9878476858139038


# Setting up All-Defense Prediction Model

In [182]:
# Setting up the data sets for each awards models
all_d_data = merged_df.copy()
all_d_data = all_d_data[(all_d_data['season'] != 2023) & (all_d_data['season'] != 2024)]
all_d_data['All-Defense'].fillna('No', inplace=True)
y = all_d_data['All-Defense']
X = all_d_data.drop(columns=['All-NBA','All Star','All-Rookie','All-Defense','season','ts_percent','x3p_ar','f_tr','orb_percent',
                             'trb_percent','ast_percent','tov_percent','usg_percent','ows','ws','obpm','bpm','fg_percent',
                             'x3pa_per_game','x3p_percent','e_fg_percent','fta_per_game','ft_percent','orb_per_game',
                             'trb_per_game','ast_per_game','tov_per_game','pts_per_game']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [183]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 16
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_107 (Dense)           (None, 8)                 136       
                                                                 
 dense_108 (Dense)           (None, 2)                 18        
                                                                 
 dense_109 (Dense)           (None, 1)                 3         
                                                                 
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________


In [184]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [185]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1042/1042 [==============================] - 1s 442us/step - loss: 0.1212 - accuracy: 0.9828
Epoch 2/100
1042/1042 [==============================] - 0s 438us/step - loss: 0.0448 - accuracy: 0.9866
Epoch 3/100
1042/1042 [==============================] - 0s 452us/step - loss: 0.0401 - accuracy: 0.9866
Epoch 4/100
1042/1042 [==============================] - 0s 431us/step - loss: 0.0388 - accuracy: 0.9866
Epoch 5/100
1042/1042 [==============================] - 0s 448us/step - loss: 0.0380 - accuracy: 0.9867
Epoch 6/100
1042/1042 [==============================] - 0s 450us/step - loss: 0.0374 - accuracy: 0.9868
Epoch 7/100
1042/1042 [==============================] - 0s 430us/step - loss: 0.0369 - accuracy: 0.9873
Epoch 8/100
1042/1042 [==============================] - 0s 428us/step - loss: 0.0366 - accuracy: 0.9869
Epoch 9/100
1042/1042 [==============================] - 0s 428us/step - loss: 0.0361 - accuracy: 0.9874
Epoch 10/100
1042/1042 [==============================]

In [186]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

348/348 - 0s - loss: 0.0335 - accuracy: 0.9878 - 194ms/epoch - 558us/step
Loss: 0.033507756888866425, Accuracy: 0.9878476858139038


# Setting up All-Rookie Prediction Model

In [143]:
# Setting up the data sets for each awards models
all_rook_data = merged_df.copy()
all_rook_data = all_rook_data[(all_rook_data['season'] != 2023) & (all_rook_data['season'] != 2024) & (all_rook_data['experience'] == 1)]
all_rook_data['All-Rookie'].fillna('No', inplace=True)
y = all_rook_data['All-Rookie']
X = all_rook_data.drop(columns=['All-NBA','All Star','All-Rookie','All-Defense','season']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [144]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 39
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_88 (Dense)            (None, 8)                 320       
                                                                 
 dense_89 (Dense)            (None, 5)                 45        
                                                                 
 dense_90 (Dense)            (None, 5)                 30        
                                                                 
 dense_91 (Dense)            (None, 1)                 6         
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [145]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [146]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
159/159 [==============================] - 0s 546us/step - loss: 0.4017 - accuracy: 0.8847
Epoch 2/100
159/159 [==============================] - 0s 500us/step - loss: 0.1835 - accuracy: 0.9178
Epoch 3/100
159/159 [==============================] - 0s 497us/step - loss: 0.1581 - accuracy: 0.9178
Epoch 4/100
159/159 [==============================] - 0s 504us/step - loss: 0.1475 - accuracy: 0.9178
Epoch 5/100
159/159 [==============================] - 0s 505us/step - loss: 0.1397 - accuracy: 0.9178
Epoch 6/100
159/159 [==============================] - 0s 502us/step - loss: 0.1286 - accuracy: 0.9390
Epoch 7/100
159/159 [==============================] - 0s 500us/step - loss: 0.1162 - accuracy: 0.9510
Epoch 8/100
159/159 [==============================] - 0s 499us/step - loss: 0.1102 - accuracy: 0.9542
Epoch 9/100
159/159 [==============================] - 0s 502us/step - loss: 0.1063 - accuracy: 0.9565
Epoch 10/100
159/159 [==============================] - 0s 505us/step - l

In [147]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

53/53 - 0s - loss: 0.1270 - accuracy: 0.9510 - 93ms/epoch - 2ms/step
Loss: 0.12701934576034546, Accuracy: 0.9510324597358704
